# BBS+ KeyGen Check

Trying to reproduce the test vector from the BBS+ draft:

IKM value `746869732d49532d6a7573742d616e2d546573742d494b4d2d746f2d67656e65726174652d246528724074232d6b6579`

Results in the SK value of `47d2ede63ab4c329092b342ab526b1079dbc2595897d4f2ab2de4d841cbe7d56`

## Algorithm from Draft

```
SK = KeyGen(IKM, key_info)

Inputs:

- IKM (REQUIRED), a secret octet string. See requirements above.
- key_info (OPTIONAL), an octet string. if this is not supplied, it
                       MUST default to an empty string.

Definitions:

- HKDF-Extract is as defined in [@!RFC5869], instantiated with hash function hash.
- HKDF-Expand is as defined in [@!RFC5869], instantiated with hash function hash.
- I2OSP and OS2IP are as defined in [@!RFC8017], Section 4.
- L is the integer given by ceil((3 * ceil(log2(r))) / 16).
- INITSALT is the ASCII string "BBS-SIG-KEYGEN-SALT-".

Outputs:

- SK, a uniformly random integer such that 0 < SK < r.

Procedure:

1. salt = INITSALT
2. SK = 0
3. while SK == 0:
4.     salt = hash(salt)
5.     PRK = HKDF-Extract(salt, IKM || I2OSP(0, 1))
6.     OKM = HKDF-Expand(PRK, key_info || I2OSP(L, 2), L)
7.     SK = OS2IP(OKM) mod r
8. return SK
```

In [ ]:
# IKM as bytes
IKM = bytes.fromhex('746869732d49532d6a7573742d616e2d546573742d494b4d2d746f2d67656e65726174652d246528724074232d6b6579')
print(IKM)
print(IKM.hex())

In [ ]:
salt = b"BBS-SIG-KEYGEN-SALT-"
print(salt)

In [ ]:
from hashlib import sha256, sha512
use_hash = sha256 # So I can easily try different hashes.

In [ ]:
# Hash the salt
m = use_hash()
m.update(salt)
hashed_salt = m.digest()
print(hashed_salt.hex())

In [ ]:
import hkdf # From local file

In [ ]:
# PRK = HKDF-Extract(salt, IKM || I2OSP(0, 1))
PRK = hkdf.hkdf_extract(hashed_salt, IKM + bytes.fromhex('00'), hash=use_hash)
print(PRK.hex())
print(len(PRK))

In [ ]:
# Check simple concatenation works...
temp_concat = IKM + bytes.fromhex('00')
print(temp_concat.hex())

In [ ]:
import math

In [ ]:
# L is the integer given by ceil((3 * ceil(log2(r))) / 16).
r = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001
L = math.ceil((3 * math.ceil(math.log2(r))) / 16)
print(L)

In [ ]:
# OKM = HKDF-Expand(PRK, key_info || I2OSP(L, 2), L)
i2oL2 = bytes.fromhex('0030') # L=48 which is 0x30
print(i2oL2[0], i2oL2[1])
OKM = hkdf.hkdf_expand(PRK, i2oL2, L, hash=use_hash)
print(OKM.hex())

In [ ]:
# To number
OKM_number = int.from_bytes(OKM, "big")
sk = OKM_number % r
print(hex(sk))